In [ ]:
import requests
from requests.structures import CaseInsensitiveDict
import pandas as pd
from pandas import DataFrame
import json
import timeit
from timeit import default_timer as timer
from datetime import timedelta
import csv
import sparql
import numpy as np

In [ ]:
#Retrieve data from the Utrecht Archives
url = "https://data.netwerkdigitaalerfgoed.nl/_api/datasets/hetutrechtsarchief/Dataset/services/Dataset/sparql"

headers = CaseInsensitiveDict()
headers["Content-Type"] = "application/x-www-form-urlencoded"

resp_list = []  
offset = -10000

for x in range(10):
  offset = offset + 10000
  queryformat =     "query=PREFIX%20dc%3A%20%3Chttp%3A%2F%2Fpurl.org%2Fdc%2Felements%2F1.1%2F%3E%0APREFIX%20dct%3A%20%3Chttp%3A%2F%2Fpurl.org%2Fdc%2Fterms%2F%3E%0APREFIX%20sw%3A%20%3Chttp%3A%2F%2Fsemanticweb.cs.vu.nl%2F2009%2F11%2Fsem%2F%3E%0APREFIX%20wd%3A%20%3Chttp%3A%2F%2Fwww.wikidata.org%2Fentity%2F%3E%0ASELECT%20%20%2A%0A%7B%0A%20%20%7B%0A%20%20%20%20select%20%3Fstreet%20%3Fyear%20%28count%20%28%3Fyear%29%20as%20%3Fcount%29%20where%0A%20%20%20%20%7B%0A%20%20%20%20%09%3Fsub%20dct%3Adate%20%3Fyear.%0A%20%20%20%20%20%20%09%3Fsub%20dct%3Aspatial%20%3Fstreet.%0A%20%20%09%09FILTER%20regex%28%3Fstreet%2C%20%22wiki%22%29%0A%09%7Dorder%20by%20%3Fyear%0A%20%20%7D%0A%7D%0A%0ALIMIT%2010000%20offset%20{}%0A%0A".format(offset)
  query = str(queryformat)
  resp = requests.post(url, headers=headers, data=query)
  respj = resp.json()
  resp_list.extend(respj)

In [ ]:
#Deduplicate and create right VALUES input format for the POST request to Wikidata
titles = list(set([item['street'] for item in resp_list]))
titles = [x.replace('http://www.wikidata.org/entity/', '') for x in titles]

In [ ]:
#Retrieving data from Wikidata
wd_endpoint = "https://query.wikidata.org/sparql"

loc_qry = """
PREFIX wikibase: <http://wikiba.se/ontology#>
PREFIX bd: <http://www.bigdata.com/rdf#>

SELECT ?street ?streetLabel ?coords ?bagID ?city ?cityLabel ?citycoords ?s_class WHERE {
VALUES ?street {wd:""" + " wd:".join([i for i in titles]) + """}
OPTIONAL {?street wdt:P625 ?coords.}
OPTIONAL {?street wdt:P5207 ?bagID . }
OPTIONAL {?street wdt:P131 ?city . }
?street wdt:P31 ?s_class .
?city wdt:P625 ?citycoords.
FILTER(?s_class = wd:Q79007 || ?s_class =  wd:Q2039348 || ?s_class = wd:Q3957)
SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],nl". }
BIND(REPLACE(str(?street), "http://www.wikidata.org/entity/", "") AS ?location)
}"""

result = sparql.query(wd_endpoint, loc_qry)
res = result.fetchall()
res = [sparql.unpack_row(row) for row in res]
bb_locs = pd.DataFrame.from_records(res, columns=result.variables)

In [ ]:
#Converting the double year notations to periods
count_df = pd.DataFrame (resp_list,columns=['street', 'year','count'])
double = {"street":[],"year":[],"count":[]}

for index, row in count_df.iterrows():
    if len(row.year) > 4:
        double["street"].append(row.street)
        double["year"].append(row.year)
        double["count"].append(row.count)
        row.year = np.nan

count_df.dropna(subset=['year'])

doubledf = DataFrame.from_dict(double)
doubledf[['start_date','end_date']] = doubledf.year.str.split("/",expand=True,)
doubledf['start_date'] = doubledf['start_date'].astype(int)
doubledf['end_date'] = doubledf['end_date'].astype(int)

extraY = {"street":[],"year":[],"count":[]}
year_range = []

for row in doubledf.itertuples():
    year_range = range(row.start_date , row.end_date+1)
    for i in year_range: 
        extraY["street"].append(row.street)
        extraY["year"].append(i)
        extraY["count"].append('1')

extraYdf = DataFrame.from_dict(extraY)

result = count_df.append(extraYdf, sort=False)
result['count'] = result['count'].astype(int)
count_result = result.groupby(['street','year']).agg(count = pd.NamedAgg(column='count', aggfunc='sum')).reset_index()

In [ ]:
#Calculate the number of data points per city per year
municipality_df = pd.merge(count_result,bb_locs[['street','city', 's_class']],on='street', how='left')
provinces = ['http://www.wikidata.org/entity/Q2039348', 'http://www.wikidata.org/entity/Q3957']
province_df = municipality_df[municipality_df.s_class.isin(provinces)]
city_df = municipality_df[~municipality_df.s_class.isin(provinces)]
province_df= province_df[['street', 'year', 'count']]
city_df = city_df[['city','year', 'count']]
province_df = province_df.rename(columns = {'street':'city'})
city_df = city_df.dropna()
all_municipalities = city_df.append(province_df, sort=False)
count_municipalities = all_municipalities.groupby(['city','year']).agg(count = pd.NamedAgg(column='count', aggfunc='sum')).reset_index()

In [ ]:
#Export to CSVs
count_municipalities.to_csv('CityCountUtrechtsArchief.csv')
count_result.to_csv("LocationCountUtrechtsArchief.csv")
bb_locs.to_csv("Wikidatalocations.csv")